In [4]:
import pandas as pd
#читаем данные в формате csv с помощью функции pd.read_csv.
train_df = pd.read_csv('train.csv')
questions_df = pd.read_csv('questions.csv')
lectures_df = pd.read_csv('lectures.csv')

# вывод первых 5 строк с помощью метода .head()
train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [6]:
questions_df.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [8]:
lectures_df.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [10]:
#объеденим train_df с questions_df по столбцу content_id из train_df и question_id из questions_df.
#Это позволяет сопоставить каждый вопрос, заданный пользователю, с его метаданными
train_questions = train_df.merge(questions_df, left_on='content_id', right_on='question_id', how='left')

In [12]:
train_questions.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
0,0,0,115,5692,0,1,3,1,NaN,NaN,5692.0,5692.0,3.0,5.0,151
1,1,56943,115,5716,0,2,2,1,37000.0,False,5716.0,5716.0,2.0,5.0,168
2,2,118363,115,128,0,0,0,1,55000.0,False,128.0,128.0,0.0,1.0,131 149 92
3,3,131167,115,7860,0,3,0,1,19000.0,False,7860.0,7860.0,0.0,1.0,131 104 81
4,4,137965,115,7922,0,4,1,1,11000.0,False,7922.0,7922.0,1.0,1.0,131 149 92


In [14]:
# Объединение train_questions(объедененный DataFrame) с lectures_df. Мы использовали content_id из train_questions и lecture_id из lectures_df
# для объединения. Это позволяет сопоставить каждую лекцию, просмотренную пользователем, с ее метаданными.
train_full = train_questions.merge(lectures_df, left_on='content_id', right_on='lecture_id', how='left', suffixes=('_question', '_lecture'))

In [16]:
train_full.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part_question,tags,lecture_id,tag,part_lecture,type_of
0,0,0,115,5692,0,1,3,1,NaN,NaN,5692.0,5692.0,3.0,5.0,151,NaN,NaN,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False,5716.0,5716.0,2.0,5.0,168,NaN,NaN,NaN,NaN
2,2,118363,115,128,0,0,0,1,55000.0,False,128.0,128.0,0.0,1.0,131 149 92,NaN,NaN,NaN,NaN
3,3,131167,115,7860,0,3,0,1,19000.0,False,7860.0,7860.0,0.0,1.0,131 104 81,NaN,NaN,NaN,NaN
4,4,137965,115,7922,0,4,1,1,11000.0,False,7922.0,7922.0,1.0,1.0,131 149 92,NaN,NaN,NaN,NaN


In [18]:
# Удаление столбцов с более чем 50% пропущенных значений
threshold = 0.5 * len(train_full)
train_full = train_full.dropna(thresh=threshold, axis=1)

In [20]:
# чтобы узнать размер таблицы используем атрибут shape 
train_full.shape 

(101230332, 15)

In [22]:
# Проверка на пропущенные значения
train_full.isnull().sum()

row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
question_id                       1191807
bundle_id                         1191807
correct_answer                    1191807
part_question                     1191807
tags                              1191808
dtype: int64

In [24]:
# Удаление столбцов с большим количеством пропусков
columns_to_drop = ['question_id', 'bundle_id', 'correct_answer', 'part_question', 'tags', 
                   'lecture_id', 'tag', 'part_lecture', 'type_of']
train_full = train_full.drop(columns=columns_to_drop, errors='ignore')

In [26]:
# чтобы узнать размер отредактированной таблицы используем атрибут shape 
train_full.shape 

(101230332, 10)

In [28]:
# Замена -1 на NaN в user_answer и answered_correctly
train_full['user_answer'] = train_full['user_answer'].replace(-1, pd.NA)
train_full['answered_correctly'] = train_full['answered_correctly'].replace(-1, pd.NA)

In [29]:
train_full.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [32]:
# Описательная статистика
train_full.describe(include='all')



,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
count,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,99271300.0,99271300.0,9.887879e+07,100837826
unique,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,True
freq,NaN,NaN,NaN,NaN,NaN,NaN,28186489.0,65244627.0,NaN,89685560
mean,5.061517e+07,7.703644e+09,1.076732e+09,5.219605e+03,1.935222e-02,9.040624e+02,NaN,NaN,2.542381e+04,NaN
std,2.922268e+07,1.159266e+10,6.197163e+08,3.866359e+03,1.377596e-01,1.358302e+03,NaN,NaN,1.994815e+04,NaN
min,0.000000e+00,0.000000e+00,1.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,NaN
25%,2.530758e+07,5.243436e+08,5.408116e+08,2.063000e+03,0.000000e+00,1.040000e+02,NaN,NaN,1.600000e+04,NaN
50%,5.061517e+07,2.674234e+09,1.071781e+09,5.026000e+03,0.000000e+00,3.820000e+02,NaN,NaN,2.100000e+04,NaN
75%,7.592275e+07,9.924551e+09,1.615742e+09,7.425000e+03,0.000000e+00,1.094000e+03,NaN,NaN,2.966600e+04,NaN


In [34]:
# Статистика по правильным ответам
train_full['answered_correctly'].value_counts(normalize=True)

answered_correctly
1    0.657236
0    0.342764
Name: proportion, dtype: float64